In [42]:
import sqlite3
import datetime as dt
import re

from textblob import TextBlob
import nltk
import pandas as pd
from dateutil import parser
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

In [43]:
df = pd.read_csv('sms.csv')
df.head(5)

,ROWID,SenderPhonNumber,Sender,text,attributedBody,type,date
0,153251,88022,Trump,Why haven't you claimed your Trump Christmas S..., streamtyped���@���NSMutableAttributedStri...,0,6.592195e+17
1,153247,88022,Trump,Don Jr: My father texted you about our new ad...., streamtyped���@���NSMutableAttributedStri...,0,6.592061e+17
2,153240,88022,Trump,Do you want a TRUMP 10X-IMPACT offer? Click th..., streamtyped���@���NSMutableAttributedStri...,0,6.591305e+17
3,153234,52005,Warnock,"￼Stacey Abrams: ""Your grassroots donations hav...", streamtyped���@���NSMutableAttributedStri...,0,6.591249e+17
4,153228,88022,Trump,NEW\nNEW\nNEW\n\nDo you have a LET'S GO BRANDO..., streamtyped���@���NSMutableAttributedStri...,0,6.591182e+17


In [44]:
df['timestamp'] = pd.to_datetime(df['date'], infer_datetime_format=True) + pd.offsets.DateOffset(years=31)
df['text'] = df['text'].replace('\\n', ' ')
df['month_name'] = df['timestamp'].dt.month_name()
df['day_name'] = df['timestamp'].dt.day_name()
df['day'] = df['timestamp'].dt.day
df['hour'] = df['timestamp'].dt.hour
df['weekday'] = df['timestamp'].dt.weekday
df['week'] = df['timestamp'].dt.isocalendar().week
df['year'] = df['timestamp'].dt.isocalendar().year

In [45]:
df = df.drop(index=[134844,134845,135206,135207,135264,135265,138476,
                   138477,138478,138479,138480,138482,138483,142979,
                   142980,144198,144199,144200])

KeyError: '[134844 134845 135206 135207 135264 135265 138476 138477 138478 138479\n 138480 138482 138483 142979 142980 144198 144199 144200] not found in axis'

In [17]:
df.head()

,ROWID,SenderPhonNumber,Sender,text,attributedBody,type,date,timestamp,month_name,day_name,day,hour,weekday,week
0,153251,88022,Trump,Why haven't you claimed your Trump Christmas S..., streamtyped���@���NSMutableAttributedString,0,659219540137591936,2021-11-21 20:32:20.137591936,November,Sunday,21,20,6,46
1,153247,88022,Trump,Don Jr: My father texted you about our new ad...., streamtyped���@���NSMutableAttributedString,0,659206057430838016,2021-11-21 16:47:37.430838016,November,Sunday,21,16,6,46
2,153240,88022,Trump,Do you want a TRUMP 10X-IMPACT offer? Click th..., streamtyped���@���NSMutableAttributedString,0,659130534492584960,2021-11-20 19:48:54.492584960,November,Saturday,20,19,5,46
3,153234,52005,Warnock,"￼Stacey Abrams: ""Your grassroots donations hav...", streamtyped���@���NSMutableAttributedString,0,659124875304199040,2021-11-20 18:14:35.304199040,November,Saturday,20,18,5,46
4,153228,88022,Trump,NEW\nNEW\nNEW\n\nDo you have a LET'S GO BRANDO..., streamtyped���@���NSMutableAttributedString,0,659118177713035904,2021-11-20 16:22:57.713035904,November,Saturday,20,16,5,46


In [6]:
blob = TextBlob(df['text'].iloc[0])
blob.noun_phrases
blob.tags
blob.sentiment
len(blob.split())

27

In [7]:
len(blob.tokenize())

36

In [8]:
blob.sentiment_assessments

Sentiment(polarity=0.19318181818181818, subjectivity=0.32727272727272727, assessments=[(['new'], 0.13636363636363635, 0.45454545454545453, None), (['legal', '!'], 0.25, 0.2, None)])

In [9]:
blob.sentiment.polarity

0.19318181818181818

In [10]:
analyzer.polarity_scores(df['text'].iloc[0])

{'neg': 0.0, 'neu': 0.683, 'pos': 0.317, 'compound': 0.9032}

In [46]:
df['polarity'] = [TextBlob(sms).sentiment.polarity for sms in df['text']]
df['subjectivity'] = [TextBlob(sms).sentiment.subjectivity for sms in df['text']]
df['negativity'] = [analyzer.polarity_scores(sms)['neg'] for sms in df['text']]
df['neutrality'] = [analyzer.polarity_scores(sms)['neu'] for sms in df['text']]
df['positivity'] = [analyzer.polarity_scores(sms)['pos'] for sms in df['text']]
df['compound'] = [analyzer.polarity_scores(sms)['compound'] for sms in df['text']]
df['nouns'] = [TextBlob(sms).noun_phrases for sms in df['text']]
df['tags'] = [TextBlob(sms).tags for sms in df['text']]
df['word_count'] = [len(TextBlob(sms).split()) for sms in df['text']]

In [47]:
nouns = []

for text in df['nouns']:
    for noun in text:
        if noun == 'corey':
            nouns.append('<recipient name>')
        elif noun == 'stop2end':
            pass 
        else:
            nouns.append(noun)
pd.Series(nouns).value_counts()

trump                            298
pres trump                       273
donate                           223
< recipient name >               221
you                              161
                                ... 
kimberly guilfoyle                 1
last end-of-month                  1
donor list > > bit.ly/3gcig0u      1
's chief                           1
finish line                        1
Length: 1287, dtype: int64

In [28]:
impact_parser = re.compile('(\d\d\d.(?<=%)|\d\d\d\d.(?<=%))|(\dX(?<=X))', re.IGNORECASE)



test = []
for text in df['text']:
    if 'impact' in text.lower():
        test.append(text)

In [22]:
for text in test:
    m = impact_parser.match(text)
    if m:
        print(m.group())

400%
400%
1000%


In [48]:
df.to_csv('ProcessedTexts.csv', index='ROWID')